In [ ]:
!pip install tweet-preprocessor

In [9]:
import pandas as pd
import preprocessor as p

from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.preprocessing.sequence import pad_sequences

# Dataset Preprocessing

## Dataset Acquisition

Dataset Tayangan Televisi

In [ ]:
df_tv = pd.read_csv('/content/sample_data/dataset_tweet_sentimen_tayangan_tv.csv')
df_tv

,Id,Sentiment,Acara TV,Jumlah Retweet,Text Tweet
0,"1,positive,HitamPutihTransTV,12,""Undang @N_Sha...",NaN,NaN,NaN,NaN
1,2,positive,HitamPutihTransTV,6.0,Selamat berbuka puasa Semoga amal ibadah hari ...
2,"3,positive,HitamPutihTransTV,9,""Ada nih di tra...",NaN,NaN,NaN,NaN
3,4,positive,HitamPutihTransTV,2.0,selamat ya mas @adietaufan masuk hitamputih
4,5,positive,HitamPutihTransTV,1.0,Asiknya nonton Hitam Putih Trans7
...,...,...,...,...,...
395,396,negative,MataNajwaMetroTV,0.0,ini apa banget deh gw paling kesel klo orang2 ...
396,397,negative,MataNajwaMetroTV,0.0,Orang miskin semakin miskin klo sekolah melaku...
397,"398,negative,MataNajwaMetroTV,0,""ga boLeh emos...",NaN,NaN,NaN,NaN
398,"399,negative,MataNajwaMetroTV,0,""dr penampilan...",NaN,NaN,NaN,NaN


In [ ]:
df_tv.dropna(inplace=True)
df_tv.reset_index(drop=True, inplace=True)
df_tv.drop(columns=['Id', 'Acara TV', 'Jumlah Retweet'], axis=1, inplace=True)
df_tv

,Sentiment,Text Tweet
0,positive,Selamat berbuka puasa Semoga amal ibadah hari ...
1,positive,selamat ya mas @adietaufan masuk hitamputih
2,positive,Asiknya nonton Hitam Putih Trans7
3,positive,@TRANS7 acara paling komplit dan menarik apala...
4,positive,hitam putih T7 inspiratif banget
...,...,...
249,negative,RT @PutryVanVan: pemimpin itu mendongrak keada...
250,negative,lembaga negara di indonesia bobrok semua... #...
251,negative,Org2 tolol. Dibilang DI DUGA dia udh mencak2. ...
252,negative,ini apa banget deh gw paling kesel klo orang2 ...


Dataset Cellular Service Provider

In [ ]:
df_cel = pd.read_csv('/content/sample_data/dataset_tweet_sentiment_cellular_service_provider.csv')
df_cel.drop(columns=['Id'], axis=1, inplace=True)
df_cel

,Sentiment,Text Tweet
0,positive,<USER_MENTION> #BOIKOT_<PROVIDER_NAME> Gunakan...
1,positive,"Saktinya balik lagi, alhamdulillah :v <PROVIDE..."
2,negative,Selamat pagi <PROVIDER_NAME> bisa bantu kenap...
3,negative,Dear <PROVIDER_NAME> akhir2 ini jaringan data ...
4,negative,Selamat malam PENDUSTA <PROVIDER_NAME>
...,...,...
295,positive,"Pantesan lancar ya, sinyal <PROVIDER_NAME> yan..."
296,positive,Alhamdulillah lancar pakai <PROVIDER_NAME>
297,positive,"Untung pakai internet <PROVIDER_NAME>, lancar,..."
298,positive,"di tempat ramai seperti di lokasi wisata, <PRO..."


Dataset Opini Film

In [ ]:
df_film = pd.read_csv('/content/sample_data/dataset_tweet_sentiment_opini_film.csv')
df_film.drop(columns=['Id'], axis=1, inplace=True)
df_film

,Sentiment,Text Tweet
0,negative,Jelek filmnya... apalagi si ernest gak mutu bg...
1,negative,Film king Arthur ini film paling jelek dari se...
2,negative,@beexkuanlin Sepanjang film gwa berkata kasar ...
3,negative,Ane ga suka fast and furious..menurutku kok je...
4,negative,"@baekhyun36 kan gua ga tau film nya, lu bilang..."
...,...,...
195,positive,Fargo juga adaptasi dari film yang cukup berha...
196,positive,637.000 waw ini sangat keren flm horor dng jum...
197,positive,@filmziarah film yang tenang dan menghanyutkan...
198,positive,Film yg amat menarik. Kisah cinta & kesetiaan ...


Dataset Pilkada

In [ ]:
df_pilkada = pd.read_csv('/content/sample_data/dataset_tweet_sentiment_pilkada_DKI_2017.csv')
df_pilkada.drop(columns=['Id', 'Pasangan Calon'], axis=1, inplace=True)
df_pilkada

,Sentiment,Text Tweet
0,negative,Banyak akun kloning seolah2 pendukung #agussil...
1,negative,#agussilvy bicara apa kasihan yaa...lap itu ai...
2,negative,Kalau aku sih gak nunggu hasil akhir QC tp lag...
3,negative,Kasian oh kasian dengan peluru 1milyar untuk t...
4,negative,Maaf ya pendukung #AgusSilvy..hayo dukung #Ani...
...,...,...
895,positive,"Kali saja bpk @aniesbaswedan @sandiuno lihat, ..."
896,positive,Kita harus dapat merangkul semua orang tanpa b...
897,positive,Ini jagoanku dibidang digital <Smiling Face Wi...
898,positive,#PesanBijak #OkeOce #GubernurGu3 ...


Joined Dataset

In [ ]:
datasets = [df_tv, df_cel, df_film, df_pilkada]

df = pd.concat(datasets)
df.reset_index(drop=True, inplace=True)
df.rename(columns={'Sentiment':'sentiment', 'Text Tweet':'tweet'}, inplace=True)
df

,sentiment,tweet
0,positive,Selamat berbuka puasa Semoga amal ibadah hari ...
1,positive,selamat ya mas @adietaufan masuk hitamputih
2,positive,Asiknya nonton Hitam Putih Trans7
3,positive,@TRANS7 acara paling komplit dan menarik apala...
4,positive,hitam putih T7 inspiratif banget
...,...,...
1649,positive,"Kali saja bpk @aniesbaswedan @sandiuno lihat, ..."
1650,positive,Kita harus dapat merangkul semua orang tanpa b...
1651,positive,Ini jagoanku dibidang digital <Smiling Face Wi...
1652,positive,#PesanBijak #OkeOce #GubernurGu3 ...


In [3]:
df = pd.read_csv('/content/sample_data/final.csv')
df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
df.rename(columns={'Sentiment':'sentiment', 'Text Tweet':'tweet'}, inplace=True)
df

,sentiment,tweet
0,1,boikot gunakan produk bangsa sendiri
1,1,saktinya balik lagi alhamdulillah v
2,0,selamat pagi bisa bantu kenapa di dalam kama...
3,0,dear akhir2 ini jaringan data lemot banget pa...
4,0,selamat malam pendusta
...,...,...
2195,0,ini apa banget deh gw paling kesel klo orang2 ...
2196,0,orang miskin semakin miskin klo sekolah melaku...
2197,0,ga boleh emosi cepat tua nonton matanajwametro...
2198,0,dr penampilan saja kyk preman taunya bkin kisr...


## Dataset Cleaning

In [ ]:
# Preparing Stopwords Corpus

stopwords = pd.read_csv('/content/sample_data/stopwords_pilkada.csv')
kamus_stopwords = list(stopwords['ada'].values)
kamus_stopwords

In [6]:
for i, j in enumerate(df['tweet']):
  df['tweet'][i] = p.clean(df['tweet'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
vec = CountVectorizer()
text_count = vec.fit_transform(df['tweet']).toarray()
text_count = pad_sequences(text_count, maxlen=29000)
text_count

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

# Model Training

## Splitting Train and Test Data

In [22]:
x_train, x_test, y_train, y_test = train_test_split(
    text_count, df['sentiment'], test_size=0.3, random_state=42
)

## Defining Model

In [23]:
MNB = MultinomialNB()
MNB.fit(x_train, y_train)

predicted = MNB.predict(x_test)
accuracy_score = metrics.accuracy_score(predicted, y_test)
print(str('Accuracy : {:04.2f}'.format(accuracy_score*100))+'%')

Accuracy : 79.70%


In [24]:
confusion_matrix(y_test, predicted)

array([[278,  67],
       [ 67, 248]])

# Sentiment Analysis

## Dataset Preprocessing

In [18]:
dataset = pd.read_json('/content/sample_data/raw_data_clean1.json')
dataset

,23322,25189,63,7171,2057,3953,640,2969,64,16178,14470,5520,26320,4951,3328,2517,4161,6480,12773,24324,26177,14107,10136,11869,716,18359,18075,898,12178,22604,6890,4293,17788,8724,6225,8688,13545,16725,5075,4782,...,8526,19446,26266,11546,4,13330,2056,7381,867,6113,12287,15986,8109,6018,1405,25453,9991,8881,8506,288,4718,16194,3501,9314,8023,9334,18753,9419,5512,7030,8804,7632,9329,12097,10408,19578,19996,23548,24290,14333
time,2021-10-13T16:32:32+00:00,2021-07-28T16:32:59+00:00,2021-12-06T04:47:57+00:00,2021-04-27T14:10:57+00:00,2021-10-15T15:32:06+00:00,2021-08-24T03:06:25+00:00,2021-11-20T05:53:35+00:00,2021-09-16T12:59:41+00:00,2021-12-06T04:43:27+00:00,2021-09-15T11:03:54+00:00,2021-10-16T11:00:21+00:00,2021-06-23T12:38:23+00:00,2021-04-30T03:24:23+00:00,2021-07-23T10:02:35+00:00,2021-09-09T01:06:20+00:00,2021-09-28T14:47:22+00:00,2021-08-19T17:36:01+00:00,2021-05-13T12:01:39+00:00,2021-11-14T09:25:34+00:00,2021-08-25T11:14:02+00:00,2021-05-16T04:15:06+00:00,2021-10-23T11:34:50+00:00,2021-04-21T11:00:00+00:00,2021-11-23T15:33:30+00:00,2021-11-18T20:50:12+00:00,2021-07-22T01:52:13+00:00,2021-07-30T11:22:47+00:00,2021-11-16T11:31:57+00:00,2021-11-18T04:00:00+00:00,2021-11-12T18:14:27+00:00,2021-05-02T03:40:01+00:00,2021-08-16T05:19:34+00:00,2021-08-07T07:17:23+00:00,2021-09-10T06:57:56+00:00,2021-05-27T07:36:15+00:00,2021-09-14T12:26:19+00:00,2021-11-06T10:05:20+00:00,2021-08-31T07:07:14+00:00,2021-07-15T14:29:39+00:00,2021-07-30T12:05:56+00:00,...,2021-10-04T04:23:07+00:00,2021-06-17T13:27:13+00:00,2021-05-04T06:20:01+00:00,2021-12-02T06:17:35+00:00,2021-12-07T17:34:00+00:00,2021-11-10T15:45:43+00:00,2021-10-15T15:36:03+00:00,2021-04-24T13:07:30+00:00,2021-11-17T00:57:58+00:00,2021-06-02T04:26:06+00:00,2021-11-17T12:42:13+00:00,2021-09-20T12:16:59+00:00,2021-11-19T03:47:38+00:00,2021-06-03T18:20:28+00:00,2021-11-10T03:33:26+00:00,2021-07-14T05:15:05+00:00,2021-04-21T19:17:02+00:00,2021-08-22T06:01:02+00:00,2021-10-06T14:59:32+00:00,2021-12-03T08:58:22+00:00,2021-07-31T13:14:53+00:00,2021-09-15T04:26:43+00:00,2021-09-02T02:50:32+00:00,2021-07-01T14:34:36+00:00,2021-11-29T17:27:49+00:00,2021-06-28T04:56:32+00:00,2021-07-09T11:46:00+00:00,2021-06-16T14:21:08+00:00,2021-06-24T00:07:34+00:00,2021-05-01T11:44:51+00:00,2021-08-31T17:57:31+00:00,2021-04-23T12:01:06+00:00,2021-06-29T02:13:57+00:00,2021-11-18T15:37:21+00:00,2021-04-21T07:08:22+00:00,2021-06-15T00:48:58+00:00,2021-05-30T13:45:57+00:00,2021-10-06T04:00:08+00:00,2021-08-28T06:48:46+00:00,2021-10-21T01:51:12+00:00
cleanTweets,[],"[a, andy, lsm, gak, lbh, dinas, daya, komnas, ...","[a, etapi, sebentar, emang, hubung, yah, femin...","[a, feminist, manifesto, chimamanda, ngozi, ad...","[a, feminist, manifesto, chimamanda, ngozi, ad...","[, a, feminist, manifesto, chimamanda, ngozi, ...","[a, feminist, manifesto, finis, chimamanda, ng...","[a, fixed, term, ok, terang, kat, definisi, fe...","[a, gara-gara, rame, feminisme, temlen, gw, ke...","[a, laki, mukul, perempuan, b, laki, yg, dukun...","[a, lil, bit, of, flashback, jepang, dulunya, ...","[a, lot, hhh, iya, sedih, bangga, redpilled, t...","[a, thread, rubi, community, community, brand,...","[aaa, dapet, ilmu, yg, suka, buku, kak, nadya,...","[aaa, samaa, kesel, banget, dikit, feminisme, ...","[aaaaa, kaburrrr, feminisme, , , ]","[aaaaaaaaaaaaaaa, gender, feminisme, univ, ]","[aada, , anarko, , feminisme]","[aah, masyak, kyk, gw, pdhl, gw, nanya, tara, ...","[aamiin, menteri, daya, perempuan, indonesia, ]","[aamiin, , minimal, jd, menteri, daya, perempuan]","[aamiin, moga, tara, gender, anggap, orang, ya...","[ab, happy, kartini, day, erina, semangat, ema...","[abad, bayak, sama, tempo, abad, tara, gender,...","[abad, feminisme, muncul, islam, tunjuk, letak...","[abad, sadar, bangsa, barat, tara, gender, sek...","[abad, western, tuh, simon, de, beauvior, an, ...","[abad, yg, islam, ude, ngajarin, cari, calon, ...","[abai, isu, sosial, seni, cipta, diskusi, , ka...","[abang, kiri, ga, dukung, daya, perempuan, yea...","[a

In [19]:
ds = pd.DataFrame(columns=['time', 'tweet'])
ds['time'] = dataset.iloc[0, 1:]
ds['tweet'] = dataset.iloc[2, 1:]
ds.reset_index(drop=True, inplace=True)
ds

,time,tweet
0,2021-07-28T16:32:59+00:00,a andy lsm gak lbh dinas daya komnas perempuan...
1,2021-12-06T04:47:57+00:00,a etapi sebentar emang hubung yah feminisme dg...
2,2021-04-27T14:10:57+00:00,a feminist manifesto chimamanda ngozi adichie ...
3,2021-10-15T15:32:06+00:00,a feminist manifesto chimamanda ngozi adichie ...
4,2021-08-24T03:06:25+00:00,a feminist manifesto chimamanda ngozi adichie ...
...,...,...
22894,2021-06-15T00:48:58+00:00,zaman tara gender aja timbang udah gitu potong...
22895,2021-05-30T13:45:57+00:00,zaman ubah tara gender capai lowong kerja buka...
22896,2021-10-06T04:00:08+00:00,zoom meeting daya perempuan
22897,2021-08-28T06:48:46+00:00,zoom meeting rangka kowsni fair online topic d...


In [25]:
vec = CountVectorizer()
count_vec = vec.fit_transform(ds['tweet']).toarray()
count_vec = pad_sequences(count_vec, maxlen=29000)

In [29]:
predicted = MNB.predict(count_vec)

In [30]:
ds['sentiment'] = predicted

for i,j in enumerate(ds['sentiment']):
  if j == 1:
    ds['sentiment'].loc[i] = 'positive'
  else:
    ds['sentiment'].loc[i] = 'negative'

ds

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,time,tweet,sentiment
0,2021-07-28T16:32:59+00:00,a andy lsm gak lbh dinas daya komnas perempuan...,positive
1,2021-12-06T04:47:57+00:00,a etapi sebentar emang hubung yah feminisme dg...,positive
2,2021-04-27T14:10:57+00:00,a feminist manifesto chimamanda ngozi adichie ...,positive
3,2021-10-15T15:32:06+00:00,a feminist manifesto chimamanda ngozi adichie ...,positive
4,2021-08-24T03:06:25+00:00,a feminist manifesto chimamanda ngozi adichie ...,positive
...,...,...,...
22894,2021-06-15T00:48:58+00:00,zaman tara gender aja timbang udah gitu potong...,negative
22895,2021-05-30T13:45:57+00:00,zaman ubah tara gender capai lowong kerja buka...,positive
22896,2021-10-06T04:00:08+00:00,zoom meeting daya perempuan,positive
22897,2021-08-28T06:48:46+00:00,zoom meeting rangka kowsni fair online topic d...,positive


In [31]:
ds.to_excel('output.xlsx')